In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage,HumanMessage
from langchain_community.chat_models import ChatOllama


In [2]:
from langgraph.graph.message import add_messages
class ChatState(TypedDict):
    messages:Annotated[list[BaseMessage], add_messages]


In [3]:
llm=ChatOllama(model="mistral:latest",
    temperature=0.2)

def chat_node(state:ChatState):

    # take user uery state
    messages=state['messages']

    # send to llm
    response=llm.invoke(messages)

    # response store state
    return {'messages':[response]}





C:\Users\vikash.vk.kumar\AppData\Local\Temp\ipykernel_36400\2102244441.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import ChatOllama``.
  llm=ChatOllama(model="mistral:latest",


In [4]:
## add memory in loop of chat
from langgraph.checkpoint.memory import MemorySaver

In [5]:
checkpointer=MemorySaver()

graph=StateGraph(ChatState)
graph.add_node('chat_node',chat_node)
graph.add_edge(START,'chat_node')
graph.add_edge('chat_node',END)

chatbot=graph.compile(checkpointer=checkpointer)




In [6]:
initial_state={
    'messages':[HumanMessage(content='What is capital of india')]
}
chatbot.invoke(initial_state)['messages'][-1].content

ValueError: Checkpointer requires one or more of the following 'configurable' keys: thread_id, checkpoint_ns, checkpoint_id

In [6]:

thread_id="1"
while True:

    user_message= input('Type here:...... ')
    print('USer:', user_message)

    if user_message.strip().lower() in ['exit','quit','bye']:
        break

    config={'configurable':{'thread_id':thread_id}}

    response=chatbot.invoke({'messages':[HumanMessage(content=user_message)]}, config=config)

    print('AI: ', response['messages'][-1].content)


USer: hii
AI:   Hello! How can I help you today? If you have any questions or need assistance with something, feel free to ask. I'm here to help!
USer: what is my name
AI:   I don't know your name as I am just a computer program and do not have the ability to remember personal information about individual users. However, if you tell me your name, I can address you by that name during our conversation. What is your name?
USer: hi what is my name 
AI:   As I mentioned earlier, I don't know your name as I am just a computer program and do not have the ability to remember personal information about individual users. If you tell me your name, I can address you by that name during our conversation. What is your name?
USer: my name is vikash 
AI:    Hello Vikash! It's nice to meet you. How can I help you today? If you have any questions or need assistance with something, feel free to ask. I'm here to help!
USer: what is my wife name
AI:    I don't know the name of your wife as I am just a com

KeyboardInterrupt: Interrupted by user

In [7]:
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END

llm = ChatOllama(
    model="mistral:latest",
    temperature=0.2
)

def chat_node(state: ChatState):
    messages = state["messages"]
    response = llm.invoke(messages)
    return {"messages": messages + [response]}

checkpointer = MemorySaver()

graph = StateGraph(ChatState)
graph.add_node("chat_node", chat_node)
graph.add_edge(START, "chat_node")
graph.add_edge("chat_node", END)

chatbot = graph.compile(checkpointer=checkpointer)

thread_id = "1"

while True:
    user_message = input("Type here: ")

    print('USer:', user_message)

    if user_message.strip().lower() in ["exit", "quit", "bye"]:
        break

    config = {"configurable": {"thread_id": thread_id}}

    response = chatbot.invoke(
        {"messages": [HumanMessage(content=user_message)]},
        config=config
    )

    print("AI:", response["messages"][-1].content)


AI:  Hello Vikash! How can I assist you today?
AI:  Your name, as per the context of our conversation, is Vikash. How can I help you further?
